# Compare antibody neutralization values
Compare neutralization values for COV2-2130 and 2130-1-0114-112 antibodies

Import Python modules:

In [1]:
import os
import pickle

import altair as alt

import pandas as pd
from functools import reduce

import yaml

Read configuration:

In [2]:
with open("config.yaml") as f:
    config = yaml.safe_load(f)

In [3]:
antibodies = ['COV2-2130','2130-1-0114-112']

Now get the averaged `polyclonal` model fits:

In [4]:
neutralization = []
for antibody in antibodies: 
    with open(os.path.join(config["escape_dir"], f"{antibody}.pickle"), "rb") as f:
        model = pickle.load(f)
    with open(os.path.join(config["escape_dir"], f"{antibody}_avg.csv"), "rb") as h:
        variants_df = pd.read_csv(h)
        variants_df['aa_substitutions'] = variants_df['mutation']
        variants_df = variants_df.filter(['aa_substitutions', 'times_seen'], axis=1)
        variants_df['antibody'] = antibody
        model.icXX(variants_df)
    neutralization.append(model.icXX(variants_df))

In [5]:
neutralization = reduce(lambda x, y: pd.merge(x, y, on = 'aa_substitutions'), neutralization)
neutralization = neutralization.loc[(neutralization['n_models_x'] >= 2) & (neutralization['n_models_y'] >= 2)]
neutralization = neutralization.loc[(neutralization['times_seen_x'] >= 3) & (neutralization['times_seen_y'] >= 3)]


Now plot the results.
We will plot the **median** IC50s across the replicate `polyclonal` fits for each antibody.
This is an interactive plot that you can mouse over for details:

In [6]:
corr_chart = (
    alt.Chart(neutralization)
    .encode(
        x=alt.X(
            "median_IC50_x",
            title="COV2-2130 IC50 (arbitrary units)",
            scale=alt.Scale(type="log"),
        ),
        y=alt.Y(
            "median_IC50_y",
            title="2130-1-0114-112 IC50 (arbitrary units)",
            scale=alt.Scale(type="log"),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if neutralization[c].dtype == float
            else c
            for c in neutralization.columns.tolist()
        ],
    )
    .mark_circle(filled=True, size=60, opacity=0.3)
    .configure_axis(grid=False)
    .resolve_scale(y="independent", x="independent")
    .properties(width=300, height=300)
)

corr_chart

alt.Chart(...)